In [1]:
import os
import re
import pandas as pd
import pdfplumber
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

In [11]:
master_df = pd.read_csv('./master.csv')
master_df = master_df.iloc[:, 1:]
master_df = master_df.round(1)
master_df = master_df.drop_duplicates().reset_index(drop=True)
print(master_df.shape)
master_df.head()

(2065, 11)


,team_name,length,width,target,goal,length_translated,width_translated,matchdate,opp_backline_num,backline_num,opponent_team_name
0,Washington Huskies,628.0,297.7,on_target,False,-43.0,-7.5,2024-09-21,4,4,Northwestern Wildcats
1,Washington Huskies,757.6,332.3,off_target,False,-82.1,-17.8,2024-09-21,4,4,Northwestern Wildcats
2,Washington Huskies,628.0,434.2,on_target,True,-43.0,-48.3,2024-09-21,4,4,Northwestern Wildcats
3,Washington Huskies,628.0,153.0,on_target,False,-43.0,35.8,2024-09-21,4,4,Northwestern Wildcats
4,Washington Huskies,674.6,434.2,on_target,False,-57.0,-48.3,2024-09-21,4,4,Northwestern Wildcats


In [ ]:
def goal_heatmap(df):

    x = np.array(x)
    y = np.array(y)

    dataNew = np.vstack([x, y])

    nbins = 20

    k = gaussian_kde(dataNew, bw_method=0.4)

    # Goal line coordinates
    goalX = [85, 85, 510, 510]
    goalY = [600, 740, 740, 600]

    # Create the plot with specific limits and aspect ratio
    plt.figure(figsize=(6, 3))
    plt.title('2D Density with shading')

    # Adjust xi and yi to match the exact plot limits
    xi, yi = np.mgrid[0:595:nbins*1j, 600:800:nbins*1j]
    zi = k(np.vstack([xi.flatten(), yi.flatten()]))

    # Plot the density with the correct limits
    plt.pcolormesh(xi, yi, zi.reshape(xi.shape), shading='gouraud', cmap='inferno')
    # cividis

    # Add goal lines
    plt.plot(goalX, goalY, color='white')

    # Set plot limits and aspect ratio 
    plt.xlim(0, 595)
    plt.ylim(600, 800)
    plt.gca().set_aspect('equal', adjustable='box')

    # plt.colorbar(label='Density')
    plt.show()


In [4]:
import numpy as np
from sklearn.cluster import DBSCAN

# Example coordinate data
x = np.array([1, 2, 2, 8, 8, 25])
y = np.array([1, 2, 3, 8, 9, 25])

def get_hot_spots(x, y):
    # Combine x and y into a single array for clustering
    coords = np.column_stack((x, y))

    # Apply DBSCAN
    epsilon = 2.5  # Maximum distance between two samples for one to be considered as in the neighborhood of the other
    min_samples = 2  # Minimum number of samples in a neighborhood for a point to be considered as a core point
    dbscan = DBSCAN(eps=epsilon, min_samples=min_samples)
    labels = dbscan.fit_predict(coords)

    # Extract cluster centers
    cluster_centers = []
    for cluster_id in set(labels):
        if cluster_id == -1:  # Skip noise points
            continue
        cluster_points = coords[labels == cluster_id]
        center = cluster_points.mean(axis=0)
        cluster_centers.append(center)

    # Separate x and y coordinates of the cluster centers
    cluster_centers = np.array(cluster_centers)
    center_x = cluster_centers[:, 0]
    center_y = cluster_centers[:, 1]

    print("Cluster centers (x, y):")
    print(cluster_centers)

    # Optional: Return x and y coordinates
    print("X coordinates of centers:", center_x)
    print("Y coordinates of centers:", center_y)

get_hot_spots(x, y)

Cluster centers (x, y):
[[1.66666667 2.        ]
 [8.         8.5       ]]
X coordinates of centers: [1.66666667 8.        ]
Y coordinates of centers: [2.  8.5]
